# Introducción a la Regresión

La regresión es una técnica de aprendizaje supervisado que se utiliza para predecir un valor continuo. A diferencia de la clasificación, donde la salida es una etiqueta categórica, en la regresión la salida es un valor numérico. Algunos ejemplos de problemas de regresión incluyen:

- Predecir el precio de una vivienda.
- Estimar la demanda de un producto.
- Determinar la puntuación de un estudiante en un examen.

### Evaluación de Modelos de Regresión

Para evaluar el rendimiento de un modelo de regresión, utilizamos varias métricas, incluyendo:

- **Error Cuadrático Medio (MSE):** Es la media de los cuadrados de los errores, es decir, la media de las diferencias cuadráticas entre los valores predichos y los valores reales.
- **Raíz del Error Cuadrático Medio (RMSE):** Es la raíz cuadrada del MSE y proporciona una medida en las mismas unidades que la variable objetivo.
- **Coeficiente de Determinación (R²):** Indica la proporción de la variabilidad en la variable objetivo que es explicada por las características del modelo. Un valor de R² cercano a 1 indica un buen ajuste del modelo.

En esta libreta, exploraremos tres métodos clásicos de regresión: Regresión Lineal, Regresión Polinómica y Regresión Ridge.

## Presentación del Dataset

Utilizaremos el conjunto de datos **California Housing**. Este dataset contiene información sobre el precio de casas en diferentes zonas de California. A continuación se describen las variables del dataset:

- **MedInc:** Ingreso medio en un bloque.
- **HouseAge:** Edad media de las casas en un bloque.
- **AveRooms:** Número promedio de habitaciones por hogar.
- **AveBedrms:** Número promedio de dormitorios por hogar.
- **Population:** Población del bloque.
- **AveOccup:** Número promedio de ocupantes por hogar.
- **Latitude:** Latitud de la ubicación del bloque.
- **Longitude:** Longitud de la ubicación del bloque.
- **MedHouseVal:** Valor medio de las casas en un bloque (variable objetivo).

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
import warnings
warnings.filterwarnings('ignore')

# Cargar el dataset de California Housing
california = fetch_california_housing()
df = pd.DataFrame(california.data, columns=california.feature_names)
df['MedHouseVal'] = california.target

# Mostrar las primeras filas del dataset
df.head()

# Preparación de Datos

Antes de aplicar cualquier algoritmo de regresión, es crucial preparar adecuadamente los datos. Este proceso incluye inspección y limpieza de datos, codificación de variables categóricas, normalización y estandarización, y finalmente, la división de los datos en conjuntos de entrenamiento y prueba.

## Inspección y Limpieza de Datos

El primer paso es inspeccionar los datos para identificar cualquier valor faltante o anomalías que puedan afectar el rendimiento del modelo. En nuestro dataset de California Housing, es importante revisar si hay valores nulos y entender la distribución de los datos.

In [ ]:
# Inspeccionar el dataset
print(df.info())
print(df.describe())

# Verificar valores nulos
print(df.isnull().sum())

## Normalización y Estandarización

La normalización y estandarización son técnicas para escalar los datos, lo cual puede mejorar el rendimiento de los modelos de regresión. 

- **Normalización:** Escala los valores de las características para que estén en un rango de 0 a 1.
- **Estandarización:** Escala los valores de las características para que tengan media 0 y desviación estándar 1.

Estandarizaremos todas las características, incluyendo latitud y longitud, para asegurar que todas las variables sean tratadas en la misma escala por los algoritmos de regresión.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Separar las características y la variable objetivo
X = df.drop('MedHouseVal', axis=1)
y = df['MedHouseVal']

# Estandarizar todas las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convertir el resultado a un DataFrame para facilitar su manipulación
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
X_scaled_df.head()

## División de los Datos

Es fundamental dividir los datos en conjuntos de entrenamiento y prueba para evaluar adecuadamente el rendimiento del modelo. Usualmente se usa una proporción de 70-30 o 80-20.

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.2, random_state=42)

# Visualizar la forma de los conjuntos de datos
print(f'Tamaño del conjunto de entrenamiento: {X_train.shape[0]} muestras')
print(f'Tamaño del conjunto de prueba: {X_test.shape[0]} muestras')

# Regresión Lineal

La regresión lineal es uno de los métodos más simples y comunes para modelar la relación entre una variable dependiente y una o más variables independientes. La idea es encontrar una línea recta (o hiperplano en dimensiones superiores) que mejor ajuste los datos.

## Parámetros del Modelo de Regresión Lineal en scikit-learn

En scikit-learn, el modelo de regresión lineal se implementa en la clase `LinearRegression`. Algunos de los parámetros importantes incluyen:

- `fit_intercept`: Indica si se debe calcular la intersección para este modelo. Si es False, la intersección no se calculará (por defecto es True).
- `normalize`: Cuando es True, las variables se normalizarán antes de realizar la regresión (por defecto es False). Este parámetro se ha desaconsejado y se recomienda estandarizar las características antes de llamar al `fit`.

## Cómo Funciona el Algoritmo

La ecuación de la regresión lineal es:

$$
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_n x_n + \epsilon
$$

donde:
- $y$ es la variable dependiente.
- $x_1, x_2, ..., x_n$ son las variables independientes.
- $\beta_0$ es la intersección.
- $\beta_1, \beta_2, ..., \beta_n$ son los coeficientes de las variables independientes.
- $\epsilon$ es el término de error.

1. **Inicialización de Parámetros:**
   El modelo comienza calculando los valores iniciales para los parámetros, incluyendo los coeficientes $\beta_0, \beta_1, ..., \beta_n$. Si `fit_intercept` es True, el modelo también calculará la intersección $\beta_0$.

2. **Cálculo de la Función de Costo:**
   La función de costo utilizada es el error cuadrático medio (MSE), que mide la media de los cuadrados de los errores (diferencias entre los valores predichos y los valores reales).

3. **Ajuste del Modelo:**
   El modelo ajusta los coeficientes $\beta$ para minimizar la función de costo, utilizando el método de mínimos cuadrados ordinarios.

4. **Predicción:**
   Una vez ajustados los coeficientes, el modelo puede realizar predicciones calculando la ecuación de la regresión lineal para nuevos valores de $x$.

### Implementación en scikit-learn

A continuación, implementaremos la regresión lineal utilizando scikit-learn.

In [ ]:
# Importar la clase LinearRegression de scikit-learn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Crear el modelo de regresión lineal
lin_reg = LinearRegression()

# Entrenar el modelo con los datos de entrenamiento
lin_reg.fit(X_train, y_train)

# Realizar predicciones con los datos de prueba
y_pred = lin_reg.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Error Cuadrático Medio (MSE): {mse:.2f}")
print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse:.2f}")
print(f"Coeficiente de Determinación (R²): {r2:.2f}")

# Visualización de las predicciones vs valores reales
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, edgecolor='k', alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=3)
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.title('Predicciones vs Valores Reales')
plt.show()

## Evaluación del Modelo

Para evaluar el rendimiento del modelo de regresión lineal, utilizamos las siguientes métricas:

- **Error Cuadrático Medio (MSE):** Es la media de los cuadrados de los errores, es decir, la media de las diferencias cuadráticas entre los valores predichos y los valores reales.
- **Raíz del Error Cuadrático Medio (RMSE):** Es la raíz cuadrada del MSE y proporciona una medida en las mismas unidades que la variable objetivo.
- **Coeficiente de Determinación (R²):** Indica la proporción de la variabilidad en la variable objetivo que es explicada por las características del modelo. Un valor de R² cercano a 1 indica un buen ajuste del modelo.

La visualización de las predicciones versus los valores reales nos ayuda a ver cómo se ajusta el modelo a los datos de prueba. Idealmente, los puntos deberían alinearse cerca de la línea de identidad (línea diagonal).

# Regresión Polinómica

La regresión polinómica es una extensión de la regresión lineal que permite modelar la relación entre la variable dependiente y las variables independientes como un polinomio. Esto es útil cuando la relación entre las variables no es lineal.

## Parámetros del Modelo de Regresión Polinómica en scikit-learn

En scikit-learn, la regresión polinómica se implementa utilizando una combinación de `PolynomialFeatures` y `LinearRegression`. Algunos de los parámetros importantes incluyen:

- `degree`: El grado del polinomio. Determina la complejidad del modelo.
- `include_bias`: Si se debe incluir un término de sesgo (intersección) en el polinomio.

## Cómo Funciona el Algoritmo

La ecuación de la regresión polinómica es:

$$
y = \beta_0 + \beta_1 x + \beta_2 x^2 + ... + \beta_n x^n + \epsilon
$$

donde:
- $y$ es la variable dependiente.
- $x$ es la variable independiente.
- $\beta_0, \beta_1, ..., \beta_n$ son los coeficientes del polinomio.
- $\epsilon$ es el término de error.

1. **Generación de Características Polinómicas:**
   Se transforman las características originales en un conjunto de características polinómicas utilizando `PolynomialFeatures`. Por ejemplo, si el grado es 2, se generan las características $x$, $x^2$, $xy$, $y^2$, etc.

2. **Ajuste del Modelo Lineal:**
   Se ajusta un modelo de regresión lineal a las características polinómicas generadas.

3. **Predicción:**
   Una vez ajustado el modelo, se pueden realizar predicciones utilizando la ecuación del polinomio ajustado.

### Implementación en scikit-learn

A continuación, implementaremos la regresión polinómica utilizando scikit-learn.

In [ ]:
# Importar las clases necesarias de scikit-learn
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

# Crear el modelo de regresión polinómica con grado 2
poly_reg = Pipeline([
    ('poly_features', PolynomialFeatures(degree=2, include_bias=False)),
    ('lin_reg', LinearRegression())
])

# Entrenar el modelo con los datos de entrenamiento
poly_reg.fit(X_train, y_train)

# Realizar predicciones con los datos de prueba
y_pred = poly_reg.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Error Cuadrático Medio (MSE): {mse:.2f}")
print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse:.2f}")
print(f"Coeficiente de Determinación (R²): {r2:.2f}")

In [ ]:
# Visualización del modelo polinómico
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, edgecolor='k', alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=3)
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.title('Predicciones vs Valores Reales (Regresión Polinómica)')
plt.show()

# Regresión con K-Nearest Neighbors (KNN)

La regresión con K-Nearest Neighbors (KNN) es un método no paramétrico utilizado para predecir valores continuos. La idea básica es que el valor predicho de una muestra es la media (o mediana) de los valores de sus k vecinos más cercanos.

## Parámetros del Modelo de Regresión KNN en scikit-learn

En scikit-learn, el modelo de regresión KNN se implementa en la clase `KNeighborsRegressor`. Algunos de los parámetros importantes incluyen:

- `n_neighbors`: Número de vecinos a utilizar para las predicciones. Es uno de los parámetros más importantes y puede afectar significativamente el rendimiento del modelo.
- `weights`: Función de ponderación utilizada en la predicción. Puede ser 'uniform' (todos los vecinos tienen el mismo peso) o 'distance' (los vecinos más cercanos tienen mayor peso).
- `algorithm`: Algoritmo utilizado para calcular los vecinos más cercanos. Puede ser 'auto', 'ball_tree', 'kd_tree', o 'brute'.
- `leaf_size`: Tamaño de las hojas pasadas al árbol BallTree o KDTree. Afecta la velocidad de construcción y consulta, así como la memoria requerida para almacenar el árbol.

## Cómo Funciona el Algoritmo

1. **Identificación de Vecinos:**
   Para una muestra de prueba, el algoritmo encuentra las k muestras de entrenamiento más cercanas en el espacio de características.

2. **Predicción:**
   La predicción se hace tomando la media (o mediana) de los valores de las k muestras más cercanas.

### Implementación en scikit-learn

A continuación, implementaremos la regresión KNN utilizando scikit-learn.

In [ ]:
# Importar la clase KNeighborsRegressor de scikit-learn
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Crear el modelo de regresión KNN
knn_reg = KNeighborsRegressor(n_neighbors=5, weights='distance', algorithm='auto')

# Entrenar el modelo con los datos de entrenamiento
knn_reg.fit(X_train, y_train)

# Realizar predicciones con los datos de prueba
y_pred = knn_reg.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Error Cuadrático Medio (MSE): {mse:.2f}")
print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse:.2f}")
print(f"Coeficiente de Determinación (R²): {r2:.2f}")

# Visualización del modelo KNN
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, edgecolor='k', alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=3)
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.title('Predicciones vs Valores Reales (Regresión KNN)')
plt.show()

# Red Neuronal para Regresión

Las redes neuronales pueden ser una herramienta poderosa para la regresión, especialmente cuando las relaciones entre las características y la variable objetivo son no lineales y complejas.

## Instalación de TensorFlow

Para implementar una red neuronal en Python, utilizaremos la biblioteca TensorFlow. Si no la tienes instalada, puedes instalarla usando el siguiente comando:

```bash
pip install tensorflow
```

## Parámetros del Modelo de Red Neuronal en Keras

En Keras, se pueden ajustar varios parámetros para construir y entrenar una red neuronal:

- `layers`: El número de capas ocultas y el número de neuronas en cada capa.
- `activation`: La función de activación para cada capa (por ejemplo, 'relu', 'sigmoid', etc.).
- `optimizer`: El algoritmo de optimización para actualizar los pesos (por ejemplo, 'adam', 'sgd', etc.).
- `loss`: La función de pérdida que se minimizará durante el entrenamiento (para regresión, típicamente 'mean_squared_error').

## Cómo Funciona una Red Neuronal

1. **Capas y Neuronas:**
   - Una red neuronal está compuesta por capas de neuronas. La primera capa es la capa de entrada, la última capa es la capa de salida, y las capas intermedias son las capas ocultas.
   - Cada neurona en una capa está conectada a todas las neuronas de la siguiente capa a través de conexiones ponderadas.

2. **Propagación Hacia Adelante:**
   - En la propagación hacia adelante, los datos de entrada se multiplican por los pesos y se pasan a través de una función de activación para producir una salida. Esta salida se convierte en la entrada de la siguiente capa.

3. **Función de Activación:**
   - Las funciones de activación introducen no linealidad en el modelo, permitiendo que la red neuronal aprenda relaciones complejas. Algunas funciones comunes son `relu` (Rectified Linear Unit) y `sigmoid`.

4. **Cálculo de la Pérdida:**
   - La pérdida es una medida de cuán lejos están las predicciones del modelo de los valores reales. Para problemas de regresión, comúnmente se usa el error cuadrático medio (MSE).

5. **Propagación Hacia Atrás (Backpropagation):**
   - En la propagación hacia atrás, la pérdida se propaga hacia atrás a través de la red, y los pesos se actualizan utilizando un algoritmo de optimización como Adam o SGD (Stochastic Gradient Descent) para minimizar la pérdida.

6. **Entrenamiento:**
   - El proceso de entrenamiento implica repetir la propagación hacia adelante y hacia atrás para múltiples épocas, ajustando los pesos cada vez para mejorar las predicciones.

### Implementación en Keras

A continuación, implementaremos una red neuronal simple utilizando Keras.

In [ ]:
# Importar las bibliotecas necesarias
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Definir el modelo de la red neuronal
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))  # Capa de entrada y primera capa oculta
model.add(Dense(32, activation='relu'))  # Segunda capa oculta
model.add(Dense(1))  # Capa de salida

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Realizar predicciones con los datos de prueba
y_pred = model.predict(X_test).flatten()

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Error Cuadrático Medio (MSE): {mse:.2f}")
print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse:.2f}")
print(f"Coeficiente de Determinación (R²): {r2:.2f}")

In [ ]:
# Visualización del modelo de Red Neuronal
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, edgecolor='k', alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=3)
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.title('Predicciones vs Valores Reales (Red Neuronal)')
plt.show()

# Visualizar el historial de entrenamiento
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Pérdida de Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de Validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.title('Historial de Entrenamiento y Validación')
plt.legend()
plt.show()

# Conclusiones

En esta libreta hemos explorado varios métodos de regresión para predecir el valor medio de las casas en California utilizando diferentes algoritmos de aprendizaje automático. A continuación, se resumen los puntos clave y conclusiones de cada uno de los métodos estudiados:

## Regresión Lineal

- **Descripción:** Es el modelo más simple y asume una relación lineal entre las características y la variable objetivo.
- **Ventajas:** Fácil de interpretar y rápido de entrenar.
- **Desventajas:** No captura relaciones no lineales complejas.
- **Rendimiento:** Este modelo sirve como una línea base para comparar con otros métodos más complejos.

## Regresión Polinómica

- **Descripción:** Extiende la regresión lineal al incluir términos polinómicos para capturar relaciones no lineales.
- **Ventajas:** Puede modelar relaciones no lineales entre características y la variable objetivo.
- **Desventajas:** Puede sobreajustarse fácilmente si se utiliza un grado polinómico alto.
- **Rendimiento:** Mejor que la regresión lineal para datos con relaciones no lineales, pero el rendimiento depende del grado del polinomio.

## Regresión con K-Nearest Neighbors (KNN)

- **Descripción:** Método no paramétrico que predice el valor de una muestra basándose en los valores de sus vecinos más cercanos.
- **Ventajas:** Simple y fácil de entender, no requiere suposiciones sobre la forma de la relación entre características y la variable objetivo.
- **Desventajas:** Puede ser computacionalmente costoso para grandes conjuntos de datos, y su rendimiento depende de la elección del parámetro k.
- **Rendimiento:** Bueno para problemas donde las relaciones entre las características son locales y no globales.

## Red Neuronal para Regresión

- **Descripción:** Modelo de aprendizaje profundo capaz de capturar relaciones no lineales complejas entre las características y la variable objetivo.
- **Ventajas:** Gran capacidad para modelar relaciones complejas y no lineales.
- **Desventajas:** Requiere más datos y poder computacional para entrenar adecuadamente. Puede ser difícil de interpretar.
- **Rendimiento:** Generalmente mejor que los métodos anteriores en capturar patrones complejos en los datos, pero su efectividad depende del diseño y entrenamiento de la red.

## Evaluación y Comparación de Modelos

Para cada uno de estos métodos, evaluamos el rendimiento utilizando métricas como el Error Cuadrático Medio (MSE), la Raíz del Error Cuadrático Medio (RMSE), y el Coeficiente de Determinación (R²). Estos modelos mostraron diferentes niveles de precisión y capacidad de generalización:

- **Regresión Lineal:** Buen rendimiento inicial, pero limitado en capturar relaciones no lineales.
- **Regresión Polinómica:** Mejor en capturar no linealidades, pero riesgo de sobreajuste.
- **KNN:** Adecuado para relaciones locales, pero sensible a la elección de k y computacionalmente costoso para grandes datasets.
- **Red Neuronal:** Alta capacidad de modelado, pero requiere ajuste cuidadoso de hiperparámetros y mayor poder computacional.

## Conclusión Final

La elección del modelo de regresión adecuado depende de la naturaleza de los datos y del problema específico. Modelos más complejos como las redes neuronales pueden ofrecer mejor rendimiento en problemas con relaciones no lineales complejas, pero también requieren más datos y poder computacional. Por otro lado, modelos más simples como la regresión lineal y polinómica pueden ser suficientes para problemas con relaciones lineales o ligeramente no lineales y son más fáciles de interpretar y entrenar.

Es fundamental probar varios modelos y ajustar sus hiperparámetros para encontrar el mejor rendimiento posible en un conjunto de datos específico. La evaluación y comparación de modelos es una parte crucial del proceso de modelado en la ciencia de datos.